# Isolating agents with dependency injection

* clean tools to remove complexity from agents
* complexity solved through dependencies injection into tools


### Decoupling Tools from Agents with `ActionContext`

When building agentic systems, tools often need access to shared resources (memory, LLM, APIs) without tightly coupling to a specific agent implementation. The **`ActionContext`** pattern provides lightweight dependency injection so tools stay **reusable, testable, and environment-agnostic**.

#### The Architectural Problem

* A single agent may **switch roles** (e.g., *developer* → *code reviewer*).
* Effective review needs **development context** (requirements, constraints, alternatives).
* Naively importing the agent or its memory inside tools creates **tight coupling**, complicates testing, and blocks reuse across agents.

---

### The `ActionContext` Pattern (Dependency Injection)

`ActionContext` is a simple container that passes **only the dependencies a tool needs** at call time.

```python
class ActionContext:
    def __init__(self, properties: Dict=None):
        self.context_id = str(uuid.uuid4())
        self.properties = properties or {}

    def get(self, key: str, default=None):
        return self.properties.get(key, default)

    def get_memory(self):
        return self.properties.get("memory", None)
```

* **Decoupled access**: Tools call `action_context.get(...)` rather than referencing global singletons or agent instances.
* **Per-request scope**: Inject request/session-specific resources (e.g., auth tokens) safely.
* **Environment control**: Swap providers in dev/test/prod without changing tool code.

---

### Before vs. After: Code Quality Review Tool

**Before (coupled & incomplete):**

```python
@register_tool(
    description="Analyze code quality and suggest improvements",
    tags=["code_quality"]
)
def analyze_code_quality(code: str) -> str:
    # Needs conversation history, but importing agent memory would couple the tool.
    return prompt_expert(
        description_of_expert="Senior software architect reviewing code quality",
        prompt=f"Review this code:\n{code}"
    )
```

**After (decoupled with `ActionContext`):**

```python
@register_tool(
    description="Analyze code quality and suggest improvements",
    tags=["code_quality"]
)
def analyze_code_quality(action_context: ActionContext, code: str) -> str:
    memory = action_context.get_memory()

    # Extract relevant development history
    development_context = []
    for mem in memory.get_memories():
        if mem["type"] == "user":
            development_context.append(f"User: {mem['content']}")
        elif mem["type"] == "assistant" and "Here's the implementation" in mem["content"]:
            development_context.append(f"Implementation Decision: {mem['content']}")

    review_prompt = f"""Review this code in the context of its development history:

Development History:
{'\n'.join(development_context)}

Current Implementation:
{code}

Analyze:
1) Does the implementation meet all stated requirements?
2) Are constraints addressed?
3) Any gaps or missed considerations?
4) Improvements within the discussed parameters?
"""
    generate_response = action_context.get("llm")
    return generate_response(review_prompt)
```

**Result:** The tool is reusable across agents; it only assumes a `memory` and an `llm` interface provided at runtime.

---

### Handling Request-Specific Dependencies (Auth, Tenancy, Feature Flags)

Inject secrets and per-request context via `ActionContext`, not globals:

```python
@register_tool(
    description="Update code review status in project management system",
    tags=["project_management"]
)
def update_review_status(action_context: ActionContext, review_id: str, status: str) -> dict:
    auth_token = action_context.get("auth_token")
    if not auth_token:
        raise ValueError("Authentication token not found in context")

    headers = {"Authorization": f"Bearer {auth_token}", "Content-Type": "application/json"}
    response = requests.post(
        f"https://...someapi.../reviews/{review_id}/status",
        headers=headers,
        json={"status": status}
    )
    if response.status_code != 200:
        raise ValueError(f"Failed to update review status: {response.text}")
    return {"status": "updated", "review_id": review_id}
```

---

### Wiring It Together in the Agent Loop

```python
def run(self, user_input: str, memory=None, action_context_props=None):
    memory = memory or Memory()

    action_context = ActionContext({
        "memory": memory,
        "llm": self.generate_response,
        # Inject request-specific properties:
        **(action_context_props or {})
    })

    while True:
        prompt = self.construct_prompt(action_context, self.goals, memory)
        response = self.prompt_llm_for_action(action_context, prompt)
        result = self.handle_agent_response(action_context, response)
        if self.should_terminate(action_context, response):
            break

# Example invocation with request-scoped auth:
some_agent.run(
    "Update the project status...",
    memory=...,
    action_context_props={"auth_token": "my_auth_token"}
)
```

---

### Why This Pattern Matters

* **Loose coupling**: Tools don’t import agent instances or global state.
* **Testability**: In unit tests, pass **fake memory**, **stub LLMs**, and **mock HTTP** via `ActionContext`.
* **Security & compliance**: Short-lived tokens live in **per-request context**, reducing blast radius.
* **Portability**: The same tool works under different orchestration frameworks—only the context wiring changes.
* **Observability**: Inject loggers, tracers, and meters without touching tool internals.

---

### Practical Tips & Guardrails

* **Define minimal interfaces** (e.g., `llm(text) -> str`, `memory.get_memories() -> list`) so tools remain framework-agnostic.
* **Validate presence** of required deps in tools (`if not action_context.get("llm"): ...`).
* **Avoid long-lived globals** for secrets or clients; construct them per request or inject pooled clients via context.
* **Redact** sensitive memory before injection when using reflection/handoff patterns.
* **Version** context properties (e.g., `llm_v2`) to migrate safely.

---

### Anti-Patterns to Avoid

* Importing the **agent** or **global memory** inside a tool.
* Storing **credentials** in module scope or as function defaults.
* Hard-coding **HTTP clients** or **endpoints** instead of injecting them.
* Mixing **orchestration logic** (planning/routing) into tool functions.

---

### Summary

`ActionContext` is a small abstraction with big payoff: it enables **dependency injection** that keeps tools **independent, composable, and secure**. By passing memory, LLM access, and request-scoped data through the context, you get flexible agent architectures that are easier to **audit, test, and evolve**—without rewriting your tools each time the runtime changes.


### Dependency Injection, the Environment, and the Decorator

When tools need shared resources (memory, auth, configs), you want to **inject** them without tightly coupling tools to a specific agent. The **`ActionContext`** pattern enables this by carrying only the dependencies a tool requests—nothing more.

#### The problem (and why DI matters)

* Many tools are **pure** (e.g., string formatting) and should not receive sensitive context.
* Some tools **require deps** (auth tokens, memory, configs).
* Pushing DI logic into the **agent loop** makes it noisy, insecure, and hard to maintain.

---

#### Keep the agent simple — push DI to the environment

Instead of branching in the agent for every tool’s needs, delegate execution to an **environment** that knows how to inject dependencies based on a tool’s signature.

```python
def handle_agent_response(self, action_context: ActionContext, response: str) -> dict:
    """Handle action without dependency management."""
    action_def, action = self.get_action(response)
    return self.environment.execute_action(self, action_context, action_def, action["args"])
```

```python
class PythonEnvironment(Environment):
    def execute_action(self, agent, action_context: ActionContext, action: Action, args: dict) -> dict:
        """Execute an action with automatic dependency injection."""
        try:
            args_copy = args.copy()

            # 1) Inject the ActionContext if tool requests it by name
            if has_named_parameter(action.function, "action_context"):
                args_copy["action_context"] = action_context

            # 2) Inject context properties when tool opts in via underscored param
            for key, value in action_context.properties.items():
                param_name = "_" + key
                if has_named_parameter(action.function, param_name):
                    args_copy[param_name] = value

            result = action.execute(**args_copy)
            return self.format_result(result)
        except Exception as e:
            return {"tool_executed": False, "error": str(e)}
```

Helper (signature check):

```python
import inspect

def has_named_parameter(func, name: str) -> bool:
    return name in inspect.signature(func).parameters
```

---

#### ActionContext recap

```python
class ActionContext:
    def __init__(self, properties: Dict=None):
        self.context_id = str(uuid.uuid4())
        self.properties = properties or {}

    def get(self, key: str, default=None):
        return self.properties.get(key, default)

    def get_memory(self):
        return self.properties.get("memory")
```

* **Per-request scope** (auth tokens, user config).
* **Swappable deps** (dev/test/prod).
* **Least privilege** (tools opt-in via param names).

---

#### Hiding dependencies from the agent (schema filtering)

Tools can declare “injected” parameters via `action_context` and **underscored** args (e.g., `_auth_token`, `_config`). The agent shouldn’t see these in the tool’s parameter schema.

```python
from typing import get_type_hints

def get_tool_metadata(func, tool_name=None, description=None, parameters_override=None, terminal=False, tags=None):
    """Extract metadata while ignoring special parameters."""
    signature = inspect.signature(func)
    type_hints = get_type_hints(func)

    args_schema = {"type": "object", "properties": {}, "required": []}

    for param_name, param in signature.parameters.items():
        if param_name in ["action_context", "action_agent"] or param_name.startswith("_"):
            continue  # hide injected deps

        # (Simplified typing handling)
        args_schema["properties"][param_name] = {"type": "string"}
        if param.default is param.empty:
            args_schema["required"].append(param_name)

    return {
        "name": tool_name or func.__name__,
        "description": description or (func.__doc__ or ""),
        "parameters": parameters_override or args_schema,
        "tags": tags or [],
        "terminal": terminal,
        "function": func
    }
```

---

#### Examples: pure vs. injected tools

**Pure tool** (no deps):

```python
@register_tool(description="Convert text to uppercase")
def to_uppercase(text: str) -> str:
    return text.upper()
```

**Injected tool** (auth + user config via `ActionContext`):

```python
@register_tool(description="Update user settings in the system")
def update_settings(action_context: ActionContext,
                    setting_name: str,
                    new_value: str,
                    _auth_token: str,
                    _user_config: dict) -> dict:
    headers = {"Authorization": f"Bearer {_auth_token}"}

    if setting_name not in _user_config["allowed_settings"]:
        raise ValueError(f"Setting {setting_name} not allowed")

    response = requests.post(
        "https://api.example.com/settings",
        headers=headers,
        json={"setting": setting_name, "value": new_value}
    )
    response.raise_for_status()
    return {"updated": True, "setting": setting_name}
```

Agent’s view (only business parameters exposed):

```python
action = {
    "tool": "update_settings",
    "args": {"setting_name": "theme", "new_value": "dark"}
}
```

Environment injects `action_context`, `_auth_token`, and `_user_config` automatically.

---

#### The decorator’s role

A `@register_tool` decorator can centralize:

* **Metadata** (name, description, tags).
* **Schema generation** (skip `action_context` & underscored params).
* **Runtime wrapper** (optionally enforce auth, logging, timeouts).

```python
def register_tool(description=None, tags=None, terminal=False, name=None):
    def wrapper(func):
        meta = get_tool_metadata(func, tool_name=name, description=description, tags=tags, terminal=terminal)
        func._tool_meta = meta  # attach metadata for the environment/registry
        return func
    return wrapper
```

---

#### Why this design works

* **Separation of concerns**: agent plans, environment injects, tools declare.
* **Least-privilege security**: tools only receive what they explicitly request.
* **Testability**: inject fakes/mocks via `ActionContext` without touching tool code.
* **Portability**: swap LLMs, memory, auth, or HTTP clients per environment.

In short: keep the agent orchestration clean, let tools *opt in* to dependencies via parameter names, and have the environment **automatically** wire them from `ActionContext`.
